In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings('ignore')
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
brc_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DA Assignments(China)/Bike Price Count by Hours/CW1_resit_dataset.csv')
brc_df

,instant,dteday,season,hr,weekday,weathersit,temp,atemp,hum,windspeed,casual,registered
0,1,2011-01-01,1.0,0,Monday,1,0.24,0.2879,0.81,0.0000,3,13
1,2,2011-01-01,NaN,1,Monday,1,0.22,0.2727,0.80,0.0000,8,32
2,3,2011-01-01,1.0,2,Monday,1,0.22,0.2727,0.80,0.0000,5,27
3,4,2011-01-01,1.0,3,Monday,1,0.24,0.2879,0.75,0.0000,3,10
4,5,2011-01-01,1.0,4,Monday,1,0.24,0.2879,0.75,0.0000,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
17374,17375,2012-12-31,1.0,19,Tuesday,2,0.26,0.2576,0.60,0.1642,11,108
17375,17376,2012-12-31,1.0,20,Tuesday,2,0.26,0.2576,0.60,0.1642,8,81
17376,17377,2012-12-31,1.0,21,Tuesday,1,0.26,0.2576,0.60,0.1642,7,83
17377,17378,2012-12-31,1.0,22,Tuesday,1,0.26,0.2727,0.56,0.1343,13,48


In [ ]:
brc_df.shape

(17379, 12)

In [ ]:
brc_df.head()

,instant,dteday,season,hr,weekday,weathersit,temp,atemp,hum,windspeed,casual,registered
0,1,2011-01-01,1.0,0,Monday,1,0.24,0.2879,0.81,0.0,3,13
1,2,2011-01-01,NaN,1,Monday,1,0.22,0.2727,0.80,0.0,8,32
2,3,2011-01-01,1.0,2,Monday,1,0.22,0.2727,0.80,0.0,5,27
3,4,2011-01-01,1.0,3,Monday,1,0.24,0.2879,0.75,0.0,3,10
4,5,2011-01-01,1.0,4,Monday,1,0.24,0.2879,0.75,0.0,0,1


In [ ]:
brc_df.tail()

,instant,dteday,season,hr,weekday,weathersit,temp,atemp,hum,windspeed,casual,registered
17374,17375,2012-12-31,1.0,19,Tuesday,2,0.26,0.2576,0.60,0.1642,11,108
17375,17376,2012-12-31,1.0,20,Tuesday,2,0.26,0.2576,0.60,0.1642,8,81
17376,17377,2012-12-31,1.0,21,Tuesday,1,0.26,0.2576,0.60,0.1642,7,83
17377,17378,2012-12-31,1.0,22,Tuesday,1,0.26,0.2727,0.56,0.1343,13,48
17378,17379,2012-12-31,NaN,23,Tuesday,1,0.26,0.2727,0.65,0.1343,12,37


In [ ]:
brc_df.dtypes

instant         int64
dteday         object
season        float64
hr              int64
weekday        object
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
casual          int64
registered      int64
dtype: object

In [ ]:
brc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     17379 non-null  int64  
 1   dteday      17379 non-null  object 
 2   season      14783 non-null  float64
 3   hr          17379 non-null  int64  
 4   weekday     17379 non-null  object 
 5   weathersit  17379 non-null  int64  
 6   temp        17379 non-null  float64
 7   atemp       17379 non-null  float64
 8   hum         15637 non-null  float64
 9   windspeed   17379 non-null  float64
 10  casual      17379 non-null  int64  
 11  registered  17379 non-null  int64  
dtypes: float64(5), int64(5), object(2)
memory usage: 1.6+ MB


In [ ]:
brc_df.isnull().sum()

instant          0
dteday           0
season        2596
hr               0
weekday          0
weathersit       0
temp             0
atemp            0
hum           1742
windspeed        0
casual           0
registered       0
dtype: int64

In [ ]:
null_values_per = brc_df.isnull().sum() / len(brc_df) * 100
null_values_per

instant        0.000000
dteday         0.000000
season        14.937568
hr             0.000000
weekday        0.000000
weathersit     0.000000
temp           0.000000
atemp          0.000000
hum           10.023592
windspeed      0.000000
casual         0.000000
registered     0.000000
dtype: float64

In [ ]:
brc_df.describe()

,instant,season,hr,weathersit,temp,atemp,hum,windspeed,casual,registered
count,17379.0000,14783.000000,17379.000000,17379.000000,17379.000000,17379.000000,15637.000000,17379.000000,17379.000000,17379.000000
mean,8690.0000,2.499831,11.546752,1.425283,0.496987,0.475775,0.626847,0.190098,35.676218,153.786869
std,5017.0295,1.107708,6.914405,0.639357,0.192556,0.171850,0.192205,0.122340,49.305030,151.357286
min,1.0000,1.000000,0.000000,1.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4345.5000,2.000000,6.000000,1.000000,0.340000,0.333300,0.480000,0.104500,4.000000,34.000000
50%,8690.0000,3.000000,12.000000,1.000000,0.500000,0.484800,0.630000,0.194000,17.000000,115.000000
75%,13034.5000,3.000000,18.000000,2.000000,0.660000,0.621200,0.780000,0.253700,48.000000,220.000000
max,17379.0000,4.000000,23.000000,4.000000,1.000000,1.000000,1.000000,0.850700,367.000000,886.000000


In [ ]:
# Create a copy of the dataframe for imputation
impute_df = brc_df.copy()

# Select columns for imputation
impute_cols = ['hum', 'season']

# Initialize KNNImputer
imputer = KNNImputer(n_neighbors=5)  # Adjust the number of neighbors as needed

# Impute missing values
imputed_values = imputer.fit_transform(impute_df[impute_cols])
imputed_df = pd.DataFrame(imputed_values, columns=impute_cols)

# Round 'hum' column to desired decimal places
decimal_places = 2  # Specify the desired number of decimal places
imputed_df['hum'] = imputed_df['hum'].round(decimal_places)

# Post-process 'season' column to limit values to 1, 2, 3, or 4
imputed_df['season'] = imputed_df['season'].round().astype(int)
imputed_df['season'] = imputed_df['season'].clip(lower=1, upper=4)

# Assign imputed values back to the original dataframe
brc_df[impute_cols] = imputed_df[impute_cols]

# Verify the imputed dataframe
print(brc_df.head())

   instant      dteday  season  hr weekday  weathersit  temp   atemp   hum  \
0        1  2011-01-01       1   0  Monday           1  0.24  0.2879  0.81   
1        2  2011-01-01       3   1  Monday           1  0.22  0.2727  0.80   
2        3  2011-01-01       1   2  Monday           1  0.22  0.2727  0.80   
3        4  2011-01-01       1   3  Monday           1  0.24  0.2879  0.75   
4        5  2011-01-01       1   4  Monday           1  0.24  0.2879  0.75   

   windspeed  casual  registered  
0        0.0       3          13  
1        0.0       8          32  
2        0.0       5          27  
3        0.0       3          10  
4        0.0       0           1  


In [ ]:
brc_df

,instant,dteday,season,hr,weekday,weathersit,temp,atemp,hum,windspeed,casual,registered
0,1,2011-01-01,1,0,Monday,1,0.24,0.2879,0.81,0.0000,3,13
1,2,2011-01-01,3,1,Monday,1,0.22,0.2727,0.80,0.0000,8,32
2,3,2011-01-01,1,2,Monday,1,0.22,0.2727,0.80,0.0000,5,27
3,4,2011-01-01,1,3,Monday,1,0.24,0.2879,0.75,0.0000,3,10
4,5,2011-01-01,1,4,Monday,1,0.24,0.2879,0.75,0.0000,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
17374,17375,2012-12-31,1,19,Tuesday,2,0.26,0.2576,0.60,0.1642,11,108
17375,17376,2012-12-31,1,20,Tuesday,2,0.26,0.2576,0.60,0.1642,8,81
17376,17377,2012-12-31,1,21,Tuesday,1,0.26,0.2576,0.60,0.1642,7,83
17377,17378,2012-12-31,1,22,Tuesday,1,0.26,0.2727,0.56,0.1343,13,48


In [ ]:
brc_df.isnull().sum()

instant       0
dteday        0
season        0
hr            0
weekday       0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
dtype: int64

In [ ]:
brc_df['count'] = brc_df['casual'] + brc_df['registered']
brc_df['count']

0         16
1         40
2         32
3         13
4          1
        ... 
17374    119
17375     89
17376     90
17377     61
17378     49
Name: count, Length: 17379, dtype: int64

In [ ]:
brc_df.head()

,instant,dteday,season,hr,weekday,weathersit,temp,atemp,hum,windspeed,casual,registered,count
0,1,2011-01-01,1,0,Monday,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,3,1,Monday,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,2,Monday,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,3,Monday,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,4,Monday,1,0.24,0.2879,0.75,0.0,0,1,1


In [ ]:
brc_df = brc_df.drop(['casual', 'registered'], axis=1)
brc_df

,instant,dteday,season,hr,weekday,weathersit,temp,atemp,hum,windspeed,count
0,1,2011-01-01,1,0,Monday,1,0.24,0.2879,0.81,0.0000,16
1,2,2011-01-01,3,1,Monday,1,0.22,0.2727,0.80,0.0000,40
2,3,2011-01-01,1,2,Monday,1,0.22,0.2727,0.80,0.0000,32
3,4,2011-01-01,1,3,Monday,1,0.24,0.2879,0.75,0.0000,13
4,5,2011-01-01,1,4,Monday,1,0.24,0.2879,0.75,0.0000,1
...,...,...,...,...,...,...,...,...,...,...,...
17374,17375,2012-12-31,1,19,Tuesday,2,0.26,0.2576,0.60,0.1642,119
17375,17376,2012-12-31,1,20,Tuesday,2,0.26,0.2576,0.60,0.1642,89
17376,17377,2012-12-31,1,21,Tuesday,1,0.26,0.2576,0.60,0.1642,90
17377,17378,2012-12-31,1,22,Tuesday,1,0.26,0.2727,0.56,0.1343,61


In [ ]:
holidays = ['2011-01-17','2011-02-21','2011-04-15','2011-05-30','2011-07-04',
            '2011-09-05','2011-10-10','2011-11-11','2011-11-24','2011-12-26',
            '2012-01-02','2012-01-16','2012-02-20','2012-04-16','2012-05-28',
            '2012-07-04','2012-09-03','2012-10-08','2012-11-12','2012-11-22',
            '2012-12-25']

# Create new column 'is_holiday'
brc_df['holiday'] = brc_df['dteday'].isin(holidays)

In [ ]:
# Drop 'dteday' column
columns_to_drop = ['dteday']
brc_df.drop(columns_to_drop, axis=1, inplace=True)

In [ ]:
brc_df['holiday']

0        False
1        False
2        False
3        False
4        False
         ...  
17374    False
17375    False
17376    False
17377    False
17378    False
Name: holiday, Length: 17379, dtype: bool

In [ ]:
brc_df['holiday'].describe()

count     17379
unique        2
top       False
freq      16879
Name: holiday, dtype: object

In [ ]:
# Perform data binning for 'hr' (hour)
hour_bins = [0, 6, 12, 18, 24]  # Example bin edges
hour_labels = ['Night', 'Morning', 'Afternoon', 'Evening']
brc_df['hr_binned'] = pd.cut(brc_df['hr'], bins=hour_bins, labels=hour_labels)

# Perform data binning for 'temp' (temperature)
temp_bins = [0, 10, 20, 30, 40]  # Example bin edges
temp_labels = ['Very Cold', 'Cold', 'Moderate', 'Hot']
brc_df['temp_binned'] = pd.cut(brc_df['temp'], bins=temp_bins, labels=temp_labels)

# Perform data binning for 'hum' (humidity)
hum_bins = [0, 30, 60, 90, 100]  # Example bin edges
hum_labels = ['Low', 'Moderate', 'High', 'Very High']
brc_df['hum_binned'] = pd.cut(brc_df['hum'], bins=hum_bins, labels=hum_labels)

# Perform data binning for 'windspeed' (wind speed)
wind_bins = [0, 10, 20, 30, 40]  # Example bin edges
wind_labels = ['Calm', 'Breeze', 'Moderate', 'Strong']
brc_df['wind_binned'] = pd.cut(brc_df['windspeed'], bins=wind_bins, labels=wind_labels)

# Display the updated dataframe
print(brc_df.head())


   instant  season  hr weekday  weathersit  temp   atemp   hum  windspeed  \
0        1       1   0  Monday           1  0.24  0.2879  0.81        0.0   
1        2       3   1  Monday           1  0.22  0.2727  0.80        0.0   
2        3       1   2  Monday           1  0.22  0.2727  0.80        0.0   
3        4       1   3  Monday           1  0.24  0.2879  0.75        0.0   
4        5       1   4  Monday           1  0.24  0.2879  0.75        0.0   

   count  holiday hr_binned temp_binned hum_binned wind_binned  
0     16    False       NaN   Very Cold        Low         NaN  
1     40    False     Night   Very Cold        Low         NaN  
2     32    False     Night   Very Cold        Low         NaN  
3     13    False     Night   Very Cold        Low         NaN  
4      1    False     Night   Very Cold        Low         NaN  


In [ ]:
brc_df

,instant,season,hr,weekday,weathersit,temp,atemp,hum,windspeed,count,holiday,hr_binned,temp_binned,hum_binned,wind_binned
0,1,1,0,Monday,1,0.24,0.2879,0.81,0.0000,16,False,NaN,Very Cold,Low,NaN
1,2,3,1,Monday,1,0.22,0.2727,0.80,0.0000,40,False,Night,Very Cold,Low,NaN
2,3,1,2,Monday,1,0.22,0.2727,0.80,0.0000,32,False,Night,Very Cold,Low,NaN
3,4,1,3,Monday,1,0.24,0.2879,0.75,0.0000,13,False,Night,Very Cold,Low,NaN
4,5,1,4,Monday,1,0.24,0.2879,0.75,0.0000,1,False,Night,Very Cold,Low,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17374,17375,1,19,Tuesday,2,0.26,0.2576,0.60,0.1642,119,False,Evening,Very Cold,Low,Calm
17375,17376,1,20,Tuesday,2,0.26,0.2576,0.60,0.1642,89,False,Evening,Very Cold,Low,Calm
17376,17377,1,21,Tuesday,1,0.26,0.2576,0.60,0.1642,90,False,Evening,Very Cold,Low,Calm
17377,17378,1,22,Tuesday,1,0.26,0.2727,0.56,0.1343,61,False,Evening,Very Cold,Low,Calm


In [ ]:
brc_df['weekday'].unique()

array(['Monday', 'Tuesday'], dtype=object)

In [ ]:
# Perform suitable encoding
# Perform one-hot encoding for categorical features
brc_df_encoded = pd.get_dummies(brc_df, columns=['weekday', 'hr_binned', 'temp_binned', 'hum_binned', 'wind_binned'])

# Encoding the "holiday" column
brc_df_encoded['holiday'] = brc_df_encoded['holiday'].astype(int)

# Dropping the "instant" column, as it is an identifier and does not contain useful information
brc_df_encoded.drop('instant', axis=1, inplace=True)

# Display the encoded DataFrame
brc_df_encoded


,season,hr,weathersit,temp,atemp,hum,windspeed,count,holiday,weekday_Monday,...,temp_binned_Moderate,temp_binned_Hot,hum_binned_Low,hum_binned_Moderate,hum_binned_High,hum_binned_Very High,wind_binned_Calm,wind_binned_Breeze,wind_binned_Moderate,wind_binned_Strong
0,1,0,1,0.24,0.2879,0.81,0.0000,16,0,1,...,0,0,1,0,0,0,0,0,0,0
1,3,1,1,0.22,0.2727,0.80,0.0000,40,0,1,...,0,0,1,0,0,0,0,0,0,0
2,1,2,1,0.22,0.2727,0.80,0.0000,32,0,1,...,0,0,1,0,0,0,0,0,0,0
3,1,3,1,0.24,0.2879,0.75,0.0000,13,0,1,...,0,0,1,0,0,0,0,0,0,0
4,1,4,1,0.24,0.2879,0.75,0.0000,1,0,1,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17374,1,19,2,0.26,0.2576,0.60,0.1642,119,0,0,...,0,0,1,0,0,0,1,0,0,0
17375,1,20,2,0.26,0.2576,0.60,0.1642,89,0,0,...,0,0,1,0,0,0,1,0,0,0
17376,1,21,1,0.26,0.2576,0.60,0.1642,90,0,0,...,0,0,1,0,0,0,1,0,0,0
17377,1,22,1,0.26,0.2727,0.56,0.1343,61,0,0,...,0,0,1,0,0,0,1,0,0,0


In [ ]:
import pandas as pd

# Assuming "brc_df" is the original DataFrame, and "brc_df_encoded" is the encoded DataFrame

# Perform one-hot encoding for categorical features in the original DataFrame
brc_df_encoded = pd.get_dummies(brc_df, columns=['weekday', 'hr_binned', 'temp_binned', 'hum_binned', 'wind_binned'])

# Encoding the "holiday" column in the original DataFrame
brc_df_encoded['holiday'] = brc_df_encoded['holiday'].astype(int)

# Dropping the "instant" column from the original DataFrame, as it is an identifier and does not contain useful information
brc_df_encoded.drop('instant', axis=1, inplace=True)

# Merging the original DataFrame with the encoded DataFrame
merged_df = pd.concat([brc_df, brc_df_encoded], axis=1)

# Display the merged DataFrame
merged_df


,instant,season,hr,weekday,weathersit,temp,atemp,hum,windspeed,count,...,temp_binned_Moderate,temp_binned_Hot,hum_binned_Low,hum_binned_Moderate,hum_binned_High,hum_binned_Very High,wind_binned_Calm,wind_binned_Breeze,wind_binned_Moderate,wind_binned_Strong
0,1,1,0,Monday,1,0.24,0.2879,0.81,0.0000,16,...,0,0,1,0,0,0,0,0,0,0
1,2,3,1,Monday,1,0.22,0.2727,0.80,0.0000,40,...,0,0,1,0,0,0,0,0,0,0
2,3,1,2,Monday,1,0.22,0.2727,0.80,0.0000,32,...,0,0,1,0,0,0,0,0,0,0
3,4,1,3,Monday,1,0.24,0.2879,0.75,0.0000,13,...,0,0,1,0,0,0,0,0,0,0
4,5,1,4,Monday,1,0.24,0.2879,0.75,0.0000,1,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17374,17375,1,19,Tuesday,2,0.26,0.2576,0.60,0.1642,119,...,0,0,1,0,0,0,1,0,0,0
17375,17376,1,20,Tuesday,2,0.26,0.2576,0.60,0.1642,89,...,0,0,1,0,0,0,1,0,0,0
17376,17377,1,21,Tuesday,1,0.26,0.2576,0.60,0.1642,90,...,0,0,1,0,0,0,1,0,0,0
17377,17378,1,22,Tuesday,1,0.26,0.2727,0.56,0.1343,61,...,0,0,1,0,0,0,1,0,0,0


In [ ]:
# Assuming "merged_df" is the DataFrame obtained after merging brc_df and brc_df_encoded

# List of columns to be removed
columns_to_remove = ['weekday', 'hr_binned', 'temp_binned', 'hum_binned', 'wind_binned']

# Remove the columns from the DataFrame for training
filtered_df = merged_df.drop(columns=columns_to_remove)

# Display the filtered DataFrame
filtered_df


,instant,season,hr,weathersit,temp,atemp,hum,windspeed,count,holiday,...,temp_binned_Moderate,temp_binned_Hot,hum_binned_Low,hum_binned_Moderate,hum_binned_High,hum_binned_Very High,wind_binned_Calm,wind_binned_Breeze,wind_binned_Moderate,wind_binned_Strong
0,1,1,0,1,0.24,0.2879,0.81,0.0000,16,False,...,0,0,1,0,0,0,0,0,0,0
1,2,3,1,1,0.22,0.2727,0.80,0.0000,40,False,...,0,0,1,0,0,0,0,0,0,0
2,3,1,2,1,0.22,0.2727,0.80,0.0000,32,False,...,0,0,1,0,0,0,0,0,0,0
3,4,1,3,1,0.24,0.2879,0.75,0.0000,13,False,...,0,0,1,0,0,0,0,0,0,0
4,5,1,4,1,0.24,0.2879,0.75,0.0000,1,False,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17374,17375,1,19,2,0.26,0.2576,0.60,0.1642,119,False,...,0,0,1,0,0,0,1,0,0,0
17375,17376,1,20,2,0.26,0.2576,0.60,0.1642,89,False,...,0,0,1,0,0,0,1,0,0,0
17376,17377,1,21,1,0.26,0.2576,0.60,0.1642,90,False,...,0,0,1,0,0,0,1,0,0,0
17377,17378,1,22,1,0.26,0.2727,0.56,0.1343,61,False,...,0,0,1,0,0,0,1,0,0,0


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming "merged_df" is the DataFrame obtained after merging brc_df and brc_df_encoded

# List of columns to be removed
columns_to_remove = ['weekday', 'hr_binned', 'temp_binned', 'hum_binned', 'wind_binned', 'instant']

# Remove the unnecessary columns for training
filtered_df = merged_df.drop(columns=columns_to_remove)

# Separate the target variable (count) from the input features
X = filtered_df.drop('count', axis=1)
y = filtered_df['count']

# Split the data into training (70%), validation (15%), and test (15%) sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.7, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Display the shape of the datasets
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


X_train shape: (5213, 34)
y_train shape: (5213, 2)
X_val shape: (6083, 34)
y_val shape: (6083, 2)
X_test shape: (6083, 34)
y_test shape: (6083, 2)


# Random Forest Regression Basic

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Assuming X_train and y_train are the training data from the previous step

# Initialize the Random Forest Regression model
rf_model = RandomForestRegressor(random_state=42)

# Fit the Random Forest Regression model to the training data
rf_model.fit(X_train, y_train)

# Make predictions on the training data using Random Forest Regression
random_forest_pred = rf_model.predict(X_train)

# Calculate the mean squared error for Random Forest Regression on the training data
mse_rf = mean_squared_error(y_train, random_forest_pred)

# Display the mean squared error for Random Forest Regression
print("Random Forest Regression MSE on Training Data:", mse_rf)


Random Forest Regression MSE on Training Data: 868.6000501618188


# Linear Regression Basic

In [ ]:

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Assuming X_train and y_train are the training data from the previous step

# Initialize the Linear Regression model
linear_model = LinearRegression()

# Fit the Linear Regression model to the training data
linear_model.fit(X_train, y_train)

# Make predictions on the training data using Linear Regression
linear_regression_pred = linear_model.predict(X_train)

# Calculate the mean squared error for Linear Regression on the training data
mse_linear = mean_squared_error(y_train, linear_regression_pred)

# Display the mean squared error for Linear Regression
print("Linear Regression MSE on Training Data:", mse_linear)


Linear Regression MSE on Training Data: 17530.31296414058


# Random Forest Regression Manual Tuning

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Assuming X_train and y_train are the training data from the previous step

# Initialize the Random Forest Regression model with manually chosen hyperparameters
rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, min_samples_split=2, min_samples_leaf=1,
                                 max_features='auto', random_state=42)

# Fit the Random Forest Regression model to the training data
rf_model.fit(X_train, y_train)

# Make predictions on the training data using Random Forest Regression
y_train_pred_rf = rf_model.predict(X_train)

# Calculate the mean squared error for Random Forest Regression on the training data
mse_rf = mean_squared_error(y_train, y_train_pred_rf)

# Display the mean squared error for Random Forest Regression
print("Random Forest Regression MSE on Training Data (Manual Tuning):", mse_rf)


Random Forest Regression MSE on Training Data (Manual Tuning): 3823.3386360122


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

# Assuming X_train and y_train are the training data from the previous step

# Initialize the Random Forest Regression model
rf_model = RandomForestRegressor(random_state=42)

# Define the hyperparameter grid for GridSearchCV
param_grid_rf = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Initialize GridSearchCV with 5-fold cross-validation
grid_search_rf = GridSearchCV(estimator=rf_model, param_grid=param_grid_rf, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit GridSearchCV to the training data
grid_search_rf.fit(X_train, y_train)

# Get the best Random Forest Regression model with optimized hyperparameters
best_rf_model = grid_search_rf.best_estimator_

# Make predictions on the training data using the best Random Forest model
y_train_pred_rf_optimized = best_rf_model.predict(X_train)

# Calculate the mean squared error for the optimized Random Forest model on the training data
mse_rf_optimized = mean_squared_error(y_train, y_train_pred_rf_optimized)

# Display the mean squared error for the optimized Random Forest model
print("Random Forest Regression MSE on Training Data (Optimized):", mse_rf_optimized)


Random Forest Regression MSE on Training Data (Optimized): 2714.3905225545127


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

# Assuming X_train and y_train are the training data from the previous step

# Initialize the Linear Regression model
linear_model = LinearRegression()

# Define the hyperparameter grid for GridSearchCV (Note: Linear Regression does not have hyperparameters, but we include an empty dictionary for consistency)
param_grid_linear = {}

# Initialize GridSearchCV with 5-fold cross-validation
grid_search_linear = GridSearchCV(estimator=linear_model, param_grid=param_grid_linear, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit GridSearchCV to the training data (Note: GridSearchCV will ignore the hyperparameter grid for Linear Regression)
grid_search_linear.fit(X_train, y_train)

# Get the best Linear Regression model (No optimization is performed as Linear Regression does not have hyperparameters)

# Make predictions on the training data using the best Linear Regression model
y_train_pred_linear_optimized = grid_search_linear.best_estimator_.predict(X_train)

# Calculate the mean squared error for the optimized Linear Regression model on the training data
mse_linear_optimized = mean_squared_error(y_train, y_train_pred_linear_optimized)

# Display the mean squared error for the optimized Linear Regression model
print("Linear Regression MSE on Training Data (Optimized):", mse_linear_optimized)


Linear Regression MSE on Training Data (Optimized): 17530.31296414058


# Baseline Ridge Regression Model:

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

# Assuming X_train and y_train are the training data from the previous step

# Initialize the Ridge Regression model
ridge_model_baseline = Ridge(alpha=1.0, random_state=42)

# Fit the Ridge Regression model to the training data
ridge_model_baseline.fit(X_train, y_train)

# Make predictions on the training data using Ridge Regression (Baseline)
y_train_pred_baseline = ridge_model_baseline.predict(X_train)

# Calculate the mean squared error for Ridge Regression (Baseline) on the training data
mse_baseline = mean_squared_error(y_train, y_train_pred_baseline)

# Display the mean squared error for Ridge Regression (Baseline)
print("Ridge Regression MSE on Training Data (Baseline):", mse_baseline)


Ridge Regression MSE on Training Data (Baseline): 17532.03776008481


#    Manual Hyperparameter Tuning for Ridge Regression:

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

# Assuming X_train and y_train are the training data from the previous step

# Initialize the Ridge Regression model
ridge_model_manual = Ridge(alpha=10, random_state=42)  # Manually chosen alpha value

# Fit the Ridge Regression model to the training data
ridge_model_manual.fit(X_train, y_train)

# Make predictions on the training data using Ridge Regression (Manual Hyperparameter Tuning)
y_train_pred_manual = ridge_model_manual.predict(X_train)

# Calculate the mean squared error for Ridge Regression (Manual Hyperparameter Tuning) on the training data
mse_manual = mean_squared_error(y_train, y_train_pred_manual)

# Display the mean squared error for Ridge Regression (Manual Hyperparameter Tuning)
print("Ridge Regression MSE on Training Data (Manual Hyperparameter Tuning):", mse_manual)


Ridge Regression MSE on Training Data (Manual Hyperparameter Tuning): 17571.641588628747


   # Automatic Hyperparameter Tuning for Ridge Regression:

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

# Assuming X_train and y_train are the training data from the previous step

# Initialize the Ridge Regression model
ridge_model = Ridge(random_state=42)

# Define the hyperparameter grid for GridSearchCV
param_grid_ridge = {
    'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],
}

# Initialize GridSearchCV with 5-fold cross-validation
grid_search_ridge = GridSearchCV(estimator=ridge_model, param_grid=param_grid_ridge, cv=5,
                                 scoring='neg_mean_squared_error', n_jobs=-1)

# Fit GridSearchCV to the training data
grid_search_ridge.fit(X_train, y_train)

# Get the best Ridge Regression model with optimized hyperparameters
best_ridge_model = grid_search_ridge.best_estimator_

# Get the best hyperparameters selected by GridSearchCV
best_alpha = best_ridge_model.alpha

# Make predictions on the training data using the best Ridge Regression model
y_train_pred_optimized = best_ridge_model.predict(X_train)

# Calculate the mean squared error for the optimized Ridge Regression model on the training data
mse_optimized = mean_squared_error(y_train, y_train_pred_optimized)

# Display the best hyperparameters selected by GridSearchCV
print("Best alpha selected by GridSearchCV:", best_alpha)

# Display the mean squared error for the optimized Ridge Regression model
print("Ridge Regression MSE on Training Data (Optimized):", mse_optimized)


Best alpha selected by GridSearchCV: 1.0
Ridge Regression MSE on Training Data (Optimized): 17532.03776008481


In [ ]:

# Make predictions on the test data using the Ridge Regression model
y_pred_ridge = best_ridge_model.predict(X_test)

# Make predictions on the test data using the Random Forest model
y_pred_rf = best_rf_model.predict(X_test)

# Calculate MSE, MAE, and R-squared for the Ridge Regression model
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
mae_ridge = mean_absolute_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)

# Calculate MSE, MAE, and R-squared for the Random Forest model
mse_rf = mean_squared_error(y_test, y_pred_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

# Display the evaluation metrics for the Ridge Regression model
print("Ridge Regression Model:")
print("MSE:", mse_ridge)
print("MAE:", mae_ridge)
print("R-squared:", r2_ridge)

# Display the evaluation metrics for the Random Forest model
print("\nRandom Forest Model:")
print("MSE:", mse_rf)
print("MAE:", mae_rf)
print("R-squared:", r2_rf)

NameError: ignored

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Assuming X_train, X_test, y_train, y_test are the training and test data from the previous steps

# Baseline Ridge Regression Model
baseline_ridge_model = Ridge(alpha=10, random_state=42)
baseline_ridge_model.fit(X_train, y_train)
y_pred_baseline_ridge = baseline_ridge_model.predict(X_test)
mse_baseline_ridge = mean_squared_error(y_test, y_pred_baseline_ridge)
mae_baseline_ridge = mean_absolute_error(y_test, y_pred_baseline_ridge)
r2_baseline_ridge = r2_score(y_test, y_pred_baseline_ridge)

# GridSearchCV optimized Ridge Regression Model
ridge_model = Ridge(random_state=42)
param_grid_ridge = {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}
grid_search_ridge = GridSearchCV(estimator=ridge_model, param_grid=param_grid_ridge, cv=5,
                                 scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_ridge.fit(X_train, y_train)
best_ridge_model = grid_search_ridge.best_estimator_
y_pred_ridge = best_ridge_model.predict(X_test)
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
mae_ridge = mean_absolute_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)

# Baseline Random Forest Model
baseline_rf_model = RandomForestRegressor(random_state=42)
baseline_rf_model.fit(X_train, y_train)
y_pred_baseline_rf = baseline_rf_model.predict(X_test)
mse_baseline_rf = mean_squared_error(y_test, y_pred_baseline_rf)
mae_baseline_rf = mean_absolute_error(y_test, y_pred_baseline_rf)
r2_baseline_rf = r2_score(y_test, y_pred_baseline_rf)

# GridSearchCV optimized Random Forest Model
rf_model = RandomForestRegressor(random_state=42)
param_grid_rf = {'n_estimators': [5,50, 100, 150], 'max_depth': [None, 10, 20, 30]}
grid_search_rf = GridSearchCV(estimator=rf_model, param_grid=param_grid_rf, cv=5,
                              scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_rf.fit(X_train, y_train)
best_rf_model = grid_search_rf.best_estimator_
y_pred_rf = best_rf_model.predict(X_test)
mse_rf = mean_squared_error(y_test, y_pred_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

# Display the evaluation metrics for the Baseline Ridge Regression model
print("Baseline Ridge Regression Model:")
print("MSE:", mse_baseline_ridge)
print("MAE:", mae_baseline_ridge)
print("R-squared:", r2_baseline_ridge)

# Display the evaluation metrics for the Optimized Ridge Regression model
print("\nOptimized Ridge Regression Model:")
print("MSE:", mse_ridge)
print("MAE:", mae_ridge)
print("R-squared:", r2_ridge)

# Display the evaluation metrics for the Baseline Random Forest model
print("\nBaseline Random Forest Model:")
print("MSE:", mse_baseline_rf)
print("MAE:", mae_baseline_rf)
print("R-squared:", r2_baseline_rf)

# Display the evaluation metrics for the Optimized Random Forest model
print("\nOptimized Random Forest Model:")
print("MSE:", mse_rf)
print("MAE:", mae_rf)
print("R-squared:", r2_rf)
